# Detection de spams avec KNN et NLP

**Le but de ce code est de dire si OUI (1) un message donné est un SPAM ou NON (0) un message entré n'est pas un SPAM pour celà nous entrainons un modèle de Random Forest sur des commentaires sous des vidéos de musique (plusieurs csv concatenés). Et pour l'aspect linguistique nous utiliserons le NLP.**

## Import des bibliothèques 

In [23]:
import pandas as pd
import numpy as np 

#Pour le modele
from sklearn.model_selection import train_test_split

# Vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

# random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


## Import et concatenation des bases

In [3]:
df = pd.concat([
        pd.read_csv('Youtube01-Psy.csv'),
        pd.read_csv('Youtube02-KatyPerry.csv'),
        pd.read_csv('Youtube03-LMFAO.csv'),
        pd.read_csv('Youtube04-Eminem.csv'),
        pd.read_csv('Youtube05-Shakira.csv'),
    ]
)

## Visualisation et analyse 

**Nous avons un dataset classifié avec des commentaires de types spam et des non spams** 

In [4]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


**Visualiser le deuxième commentaire**

In [5]:
df.CONTENT.iloc[2]

'just for test I have to say murdev.com'

**Visualiser le nombre de spam (1) et non spam (0).**

In [6]:
df.CLASS.value_counts()

CLASS
1    1005
0     951
Name: count, dtype: int64

**En pourcentage**

In [8]:
df.CLASS.value_counts(normalize=True)

CLASS
1    0.513804
0    0.486196
Name: proportion, dtype: float64

## Spliter notre dataset en Train/Test

**Random seed permet de garantir la reproductabilité (équitable en 0 et 1)**

In [10]:
seed = 120 
X_train, X_test, Y_train, Y_test = train_test_split(
    df['CONTENT'],
    df['CLASS'],
    test_size=0.2,
    random_state = seed
)


**Ce n'est pas notre cas maintenant mais si les shapes ne sont pas pareil il faut l'ajuster (surtout dans le cas de regression)**

In [11]:
X_train.shape

(1564,)

In [12]:
Y_train.shape

(1564,)

## La vectorisation est la representation numérique du contenu de chaque commentaire

In [21]:
# Création de l'objet vectorizer
vectorizer = CountVectorizer()

# Transformer les données d'entrainnement en une valeure numérique 
X_train_vec = vectorizer.fit_transform(X_train)

# Transformer les données de test en une valeure numérique 
X_test_vec = vectorizer.transform(X_test)

#Afficher la representation numerique du train
X_train_vec

<1564x3802 sparse matrix of type '<class 'numpy.int64'>'
	with 20752 stored elements in Compressed Sparse Row format>

## Import et entrainement des données sur le modèle Random Forest

In [30]:
# Créer un modèle de Random Forest
model = RandomForestClassifier(random_state=seed)

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train_vec, Y_train)

# Prédire les étiquettes des données de test
Y_pred = model.predict(X_test_vec)

# Calculer l'accuracy du modèle
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy de Random Forest :", accuracy)



Accuracy de Random Forest : 0.9438775510204082


**Cross validation**

In [31]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95       193
           1       0.98      0.91      0.94       199

    accuracy                           0.94       392
   macro avg       0.95      0.94      0.94       392
weighted avg       0.95      0.94      0.94       392



**Predir une phrase grâce au modèle**

In [33]:
Commentaire = [["Waaaw beautiful singing!"],["What the heck bro is high as"],["Give me fu*king money"],["It is a beautiful day"]]

for com in Commentaire: 
    com_vec = vectorizer.transform(com)
    print(model.predict(com_vec))
    

[0]
[0]
[1]
[0]


**Disclaimer: ce modèle n'est pas assez robuste pour moderer dans un site de type fb, youtube...etc.**

## On peut modifier les hyperparamètres (n_estimato,max depth...etc)

**Les hyperparametres permettent d'augmenter la robustesse du modele**

In [40]:
# Créer un modèle de Random Forest
model = RandomForestClassifier(random_state=seed,n_estimators=5,max_depth=80)

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train_vec, Y_train)

# Prédire les étiquettes des données de test
Y_pred = model.predict(X_test_vec)

# Calculer l'accuracy du modèle
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy de Random Forest :", accuracy)

Accuracy de Random Forest : 0.9311224489795918


In [41]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93       193
           1       0.95      0.91      0.93       199

    accuracy                           0.93       392
   macro avg       0.93      0.93      0.93       392
weighted avg       0.93      0.93      0.93       392



In [44]:
Commentaire = [["Waaaw beautiful singing!"],["What the heck bro is high as"],["Give me freaking money"],["It is a beautiful day"]]

for com in Commentaire: 
    com_vec = vectorizer.transform(com)
    print(model.predict(com_vec))

[0]
[0]
[1]
[0]
